In [1]:
"""
Purpose: To look at the initial stats of the 
graph from autoproofreading


"""

'\nPurpose: To look at the initial stats of the \ngraph from autoproofreading\n\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import sys
sys.path.append("/meshAfterParty/meshAfterParty")
sys.path.append("/meshAfterParty/meshAfterParty/graph/")

In [4]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-11 07:37:10,591 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-11 07:37:10,593 - settings - Setting database.user to celiib
INFO - 2021-03-11 07:37:10,594 - settings - Setting database.password to newceliipass
INFO - 2021-03-11 07:37:10,599 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-11 07:37:10,600 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-11 07:37:10,614 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-11 07:37:10,946 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-11 07:37:10,950 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-11 07:37:10,951 - settings - Setting database.user to celiib
INFO - 2021-03-11 07:37:10,951 - settings - Setting database.password to newceliipass
INFO - 2021-03-11 07:37:10,952 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-11 07:37:10,952 - settings - Setting database.user to celiib
INFO - 2021-03-11 07:37:10,953 - settings - Setting database.password to newceliipass
INFO - 2021-03-11 07:37:10,955 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-11 07:37:11,313 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-11 07:37:11,346 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-11 07:37:11,347 - settings - Setting database.user to celiib
INFO - 2021-03-11 07:37:11,348 - settings - Setting database.pa

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu
import numpy_utils as nu
from importlib import reload

In [6]:
import graph_visualizations as gviz
import graph_analysis as ga

# Getting the Statistics on the Initial Graph

In [8]:
minnie.AutoProofreadNeurons3.describe()

-> minnie.Decomposition
split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
-> minnie.AutoProofreadVersion
---
multiplicity         : tinyint unsigned             # the total number of neurons that came from the parent segment id
cell_type_predicted  : enum('excitatory','inhibitory','other','unknown') # morphology predicted by classifier
spine_category       : enum('no_spined','sparsely_spined','densely_spined') 
n_axons              : tinyint unsigned             # Number of axon candidates identified
n_apicals            : tinyint unsigned             # Number of apicals identified
axon_length          : double                       # length (in um) of the classified axon skeleton
axon_area            : double                       # # area (in um^2) of the classified axon
nucleus_id           : int unsigned                 # id of nucleus from the flat segmentation  Equivalent to Allen: 'id'.
nuclei_distanc

"-> minnie.Decomposition\nsplit_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM\n-> minnie.AutoProofreadVersion\n---\nmultiplicity         : tinyint unsigned             # the total number of neurons that came from the parent segment id\ncell_type_predicted  : enum('excitatory','inhibitory','other','unknown') # morphology predicted by classifier\nspine_category       : enum('no_spined','sparsely_spined','densely_spined') \nn_axons              : tinyint unsigned             # Number of axon candidates identified\nn_apicals            : tinyint unsigned             # Number of apicals identified\naxon_length          : double                       # length (in um) of the classified axon skeleton\naxon_area            : double                       # # area (in um^2) of the classified axon\nnucleus_id           : int unsigned                 # id of nucleus from the flat segmentation  Equivalent to Allen: 'id'.\nn

In [ ]:
orig_presyn,orig_postsyn = (du.m65mat.NucleusSynapse() & "presyn != postsyn").fetch("presyn","postsyn")

In [ ]:
original_edges = np.vstack([orig_presyn,orig_postsyn]).T
original_edges

# Function that will do a lot of the analysis for a graph and return results

# a) Original Graph (Directed and Multi-Graph)

In [ ]:
G_orig_multi_di = nx.MultiDiGraph()
_ = G_orig_multi_di.add_edges_from(original_edges)

In [ ]:
graph_title = "Original Graph (Multi-Edge,Diretional)"

In [ ]:
print(f"Stats for Original Graph (Multi-Edge, Directional)")
print(f"Number of Nodes = {nu.comma_str(len(G_orig_multi_di.nodes()))}")
print(f"Number of Edges = {nu.comma_str(G_orig_multi_di.size())}")

In [ ]:
degree_distribution_filtered_orig_multi_di = ga.degree_distribution(G_orig_multi_di,
                                                     percentile=99.5)
mean_orig_multi_di = np.mean(degree_distribution_filtered_orig_multi_di)
median_orig_multi_di = np.median(degree_distribution_filtered_orig_multi_di)
print(f"mean_orig_multi_di = {nu.comma_str(mean_orig_multi_di)}, median_orig_multi_di = {nu.comma_str(median_orig_multi_di)}")

In [ ]:
degree_type_list = ["in_and_out","in","out"]

for k in degree_type_list:
    gviz.plot_degree_distribution(G_orig_multi_di,
                                 degree_type=k,
                                 title_append=graph_title,
                                 percentile=99.5)

# For the Undirected Type

In [ ]:
verbose=True

In [ ]:
G_orig_multi = nx.MultiGraph(G_orig_multi_di)

In [ ]:
graph_title = "Original Graph (Multi-Edge, Undirectional)"

In [ ]:
print(f"Stats for {graph_title}")
print(f"Number of Nodes = {nu.comma_str(len(G_orig_multi.nodes()))}")
print(f"Number of Edges = {nu.comma_str(G_orig_multi.size())}")

In [ ]:
# getting the biggest connected component
curr_graph_larg_comp = ga.largest_connected_component(G_orig_multi)
print(f"Largest Component of {graph_title}:")
print(f"Number of Nodes = {nu.comma_str(len(curr_graph_larg_comp.nodes()))}")
print(f"Number of Edges = {nu.comma_str(len(curr_graph_larg_comp.edges()))}")

In [ ]:
degree_type_list = ["in_and_out","in","out"]

G = G_orig_multi

for k in degree_type_list:
    try:
        curr_degree_distr = ga.degree_distribution(G,
                                                   degree_type=k,
                                                    percentile=99.5)
    except:
        continue
        
    curr_degree_distr_mean = np.mean(curr_degree_distr)
    curr_degree_distr_median = np.median(curr_degree_distr)
    
    if verbose:
        print(f"{graph_title} {k} degree distribution mean = {nu.comma_str(curr_degree_distr_mean)},\n"
              f"{graph_title} {k} degree distribution median = {nu.comma_str(curr_degree_distr_median)}")


    gviz.plot_degree_distribution(G,
                                 degree_type=k,
                                 title_append=graph_title,
                                  degree_distribution=curr_degree_distr,
                                 print_degree_distribution_stats=False)

In [ ]:
outptu_dict = ga.graph_analysis_basic_with_components(G=G_orig_multi_di,
                        graph_title = "Original Graph (Multi-Edge,Diretional)",
                        verbose = True,
                        plot_distributions = True,
                        degree_distribution_percentile=99.5)
    

In [ ]:
outptu_dict

In [ ]:
ga.degree_distribution(G_orig_multi_di,
                                                       degree_type="in",
                                                        percentile=99.5)

In [ ]:
# degree_distribution_analysis(G = G_orig_multi)
ga.degree_distribution_analysis(G = G_orig_multi_di)


# Original Graph but undirected non-multi-edge (simple)

In [ ]:
G_orig = nx.Graph(G_orig_multi_di)

In [ ]:
print(f"Stats for Original Graph simple (Non-Multi-Edge, Undirectional)")
print(f"Number of Nodes = {nu.comma_str(len(G_orig.nodes()))}")
print(f"Number of Edges = {nu.comma_str(G_orig.size())}")

In [ ]:
# The Sizes of the components in the graph
conn_comp = list(nx.connected_components(G_undirected_orig))
conn_comp_size = [len(k) for k in conn_comp]
print(f"conn_comp_size = {conn_comp_size}")

# Pull Down and Calculating the Initial Graph

In [ ]:
def fetch_graph_proofread(apply_non_limb_presyn_filter=True):
    """
    Purpose: To get the 
    
    
    """
#subtracting off presyns that are not on limb branches 
if apply_non_limb_presyn_filter:
    table_to_subtract = (du.proofreading_synapse_table() & "skeletal_distance_to_soma<0 AND synapse_type='presyn'").proj()
    synapse_data = (du.proofreading_synapse_table() - table_to_subtract).fetch(as_dict=True)
else:
    synapse_data = du.proofreading_synapse_table() 

# pulling down the data
synapse_data_df = pd.DataFrame.from_dict(synapse_data)

#This function convert a dataframe that stores presyn and postsyn into a dataframe of 


In [ ]:
""

In [ ]:
uniq_synapse_ids,unique_synapse_indexes,inverse_indices,synapse_id_counts = np.unique(synapse_data_df["synapse_id"].to_numpy(),
                           return_index=True,
                           return_inverse=True,
                            return_counts=True)

In [ ]:
direct_connection_synapse_idx = np.where(synapse_id_counts==2)[0]
direct_connection_synapse_mask = np.zeros(len(synapse_id_counts))
direct_connection_synapse_mask[direct_connection_synapse_idx] = 1
direct_connection_synapse_mask = direct_connection_synapse_mask.astype("bool")
row_filter = direct_connection_synapse_mask[inverse_indices]

direct_connections_df = synapse_data_df.iloc[row_filter]
direct_connections_df

In [ ]:
# Combining the table's presyns and postsyns onto the same row

df_tables = []
for synapse_type in ["presyn","postsyn"]:
    direct_connections_df_init = direct_connections_df[direct_connections_df["synapse_type"]==synapse_type]
    
    direct_connections_df_init_sorted = direct_connections_df_init.sort_values(by=['synapse_id']).reset_index(drop=True)
    
    direct_connections_df_presyn = direct_connections_df_init_sorted[[k for k in direct_connections_df_init_sorted.columns if k != "synapse_type"] ]
    
    #column_mapper = dict([(k,f"{k}_{synapse_type}") if k != "synapse_id"  else (k,k) for k in direct_connections_df_presyn.columns])
    column_mapper = dict([(k,f"{synapse_type}_{k}") for k in direct_connections_df_presyn.columns])

    presyn_df = direct_connections_df_presyn.rename(columns=column_mapper)
    df_tables.append(presyn_df)
    
direct_conn_df = pd.concat(df_tables,axis=1)
direct_conn_df

In [ ]:
edges_df = direct_conn_df[["presyn_nucleus_id","postsyn_nucleus_id"]]
connectome_edges = edges_df.to_numpy()
connectome_edges

# Preliminary Look at the Graph

In [ ]:
import networkx as nx

In [ ]:
G = nx.MultiDiGraph()
G.add_edges_from(connectome_edges)
len(G.nodes()),len(G.edges())

In [ ]:
G_undirected_non_multi = nx.Graph(G)
len(G_undirected_non_multi.nodes()),len(G_undirected_non_multi.edges())

In [ ]:
G_directed_non_multi = nx.DiGraph(G)
len(G_directed_non_multi.nodes()),len(G_directed_non_multi.edges())

In [ ]:
#For connected components cannot be a directed graph
G_undirected = G_undirected_non_multi

n_components = nx.number_connected_components(G_undirected)
print(f"Number of connected components = {n_components}")

# The Sizes of the components in the graph
conn_comp = list(nx.connected_components(G_undirected))
conn_comp_size = [len(k) for k in conn_comp]
print(f"conn_comp_size = {conn_comp_size}")

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(conn_comp_size)
ax.set_title("Number of Neurons in Connected Components")
ax.set_xlabel("Number of Neurons")
plt.show()

# Selecting the largest component

In [ ]:
G_largest = G.subgraph(conn_comp[0])
node_names_largest_comp = np.array(G_largest.nodes())
node_names_largest_comp.shape

# Plotting the Degree Distribution

In [ ]:
import graph_visualizations as gu

In [ ]:
G.nodes

In [ ]:
gu.plot_degree_distribution(G,logscale=True,n_bins = 100,percentile=99,
                           font_size=20)

In [ ]:
gu.plot_degree_distribution(G,logscale=True,n_bins = 100,percentile=99,
                           font_size=18,
                           title="Degree Distribution v3 \n Neurons with Axon Length > 1000")

# Getting the statistics on the graph

In [ ]:
large_axon_nuclei = (du.proofreading_neurons_table() & "axon_length>1000").fetch("nucleus_id")
large_axon_nuclei.shape
large_axon_subgraph = G.subgraph(large_axon_nuclei)


In [ ]:
gu.plot_degree_distribution(large_axon_subgraph,logscale=True,n_bins = 100,percentile=99,
                           font_size=18,
                           title="Degree Distribution v3 \n Neurons with Axon Length > 1000")

In [ ]:
print(f"""
Graph Statistics:

Within the Direct Connection Graph
1) {len(G.nodes())} Nodes
2) {len(G.edges())} Direct Connections
3) {len(large_axon_subgraph.nodes())} Nodes with Axon Skeletal Length > 1000
4) {len(large_axon_subgraph.edges())} Direct Connections between Nodes with Axon Skeletal Length > 1000

""")

In [ ]:
excitatory_nuclei = (du.proofreading_neurons_table() & dict(spine_category = "densely_spined")).fetch("nucleus_id")
excitatory_nuclei
G_exc = G.subgraph(excitatory_nuclei)
large_axon_nuclei_exc =  (du.proofreading_neurons_table() & dict(spine_category = "densely_spined")
                                             & "axon_length>1000").fetch("nucleus_id")
large_axon_subgraph_exc = G.subgraph(large_axon_nuclei_exc)
                              

In [ ]:
minnie.AutoProofreadNeurons3()

In [ ]:
print(f"""
Graph Statistics:

* For an all densely spined excitatory graph *
Within the Direct Connection Graph
1) {len(G_exc.nodes())} Nodes
2) {len(G_exc.edges())} Direct Connections
3) {len(large_axon_subgraph_exc.nodes())} Nodes with Axon Skeletal Length > 1000
4) {len(large_axon_subgraph_exc.edges())} Direct Connections between Nodes with Axon Skeletal Length > 1000

""")

In [ ]:
curr_graph = orig_G

print(f"""
Graph Statistics:

* For an all densely spined excitatory graph *
Within the Direct Connection Graph
1) {len(orig_G.nodes())} Nodes
2) {len(orig_G.edges())} Direct Connections
3) {len(G_undirected_orig)}: Undirected, non-multi_edge Direct Connections

""")

In [ ]:
orig_G = nx.MultiDiGraph()
orig_G.add_edges_from(original_edges)
print(len(orig_G.nodes()),len(orig_G.edges()))

G_undirected_orig = nx.Graph(orig_G)
print(len(G_undirected_orig.nodes()),len(G_undirected_orig.edges()))

In [ ]:
"""
Steps for Decomposition:

Preprocessing:
1) Auto Glia and Nuclei Removal

Decomposition:
1) Find Soma
2) Divide into Soma and Branches
3) Skeletonize All Branches
4) Create Concept Graphs of all Limbs
5) Automatic Spine Detection, Width calculation

Proofreading:
6) Split Multi-Nuclei Cells
7) Axon and Apical Identification
8) Excitatory/Inhibitory Classification
9) E/I based mesh filtering away
10) Synapse Filtering based on mesh filtering away




"""

# Table that will compute the graph stats

# Degree Distribution

In [ ]:
undirectional_graph = nx.Graph(G_largest)
no_selflooped_graph = xu.remove_selfloops(undirectional_graph)
no_selflooped_graph

In [ ]:
degree_distribution = np.array(xu.get_node_degree(no_selflooped_graph,node_names_largest_comp))
degree_distribution_filtered = degree_distribution[degree_distribution<np.percentile(degree_distribution,99.5)]

In [ ]:
print(f"Average Degree of Node: {np.mean(degree_distribution_filtered)}")
print(f"Median Degree of Node: {np.median(degree_distribution_filtered)}")

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(degree_distribution_filtered,bins=50,density=True)
ax.set_title("Degree Distribution of Neurons \n Unique Edges, No Self Loops")
ax.set_xlabel("Degree of Neuron")
ax.set_ylabel("Density")
plt.show()

# Degree Distribution In and Out

In [ ]:
np.sum(degree_distribution(G,degree_type="in")),np.sum(degree_distribution(G,degree_type="out"))

In [ ]:
np.sum(degree_distribution(G,degree_type="in_and_out"))

In [ ]:
import graph_analysis as ga
import graph_visualizations as gviz

In [ ]:
from importlib import reload
gviz = reload(gviz)

In [ ]:
in_degree = np.array(list(dict(G.in_degree()).values()))
in_degree

In [ ]:
import matplotlib.pyplot as plt
import matplotlib_utils as mu

import numpy as np

mu.histogram(in_degree,bin_max = 700,
         bin_width = 10,return_fig_ax=True,logscale=True)

In [ ]:
ga.degree_distribution(G,
                                                       degree_type="in",
                                                       percentile=95)

In [ ]:
gviz.plot_degree_distribution(G,degree_type="in",
                             percentile=100,
                              bin_width = 10,
                              bin_max = 600,
                             )

In [ ]:
gviz.plot_degree_distribution(G,degree_type="out",
                             percentile=100,
                              bin_width = 10,
                              bin_max = 600,
                             )

In [ ]:
gviz.plot_degree_distribution(G,degree_type="in",
                         bin_width=5,
                         bin_max = 800,
                        percentile=99)

In [ ]:
gviz.plot_degree_distribution(G,degree_type="out",
                         bin_width=5,
                         bin_max = 800,
                        percentile=99)

In [ ]:
gviz.plot_degree_distribution(G,degree_type="in",title="In-Degree Distribution",
                         bin_width=5,
                         bin_max = 500,
                        percentile=99)

In [ ]:
plot_degree_distribution(G,degree_type="out",title="Out-Degree Distribution",
                        percentile=99)

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(degree_distribution_filtered,bins=50,density=False)
ax.set_title("Degree Distribution of Neurons \n Unique Edges, No Self Loops")
ax.set_xlabel("Degree of Neuron")
ax.set_ylabel("Count")
ax.set_yscale("log")
plt.show()

# Getting the Soma to Soma Distance

In [ ]:
"""
Psuedoode: Pull down the Nucleus IDs and Nucleus Centers

1) Get the nucleus ids and centers
2) Adjust the centers for nm


"""

nuc_table = du.configure_nucleus_table() & "segment_id>0"
nuc_table

In [ ]:
nucleu_ids,nuc_x,nuc_y,nuc_z = nuc_table.fetch("nucleus_id","nucleus_x","nucleus_y","nucleus_z")

In [ ]:
nucleus_centers = np.vstack([nuc_x,nuc_y,nuc_z]).T * [4,4,40]
nucleus_centers

In [ ]:
nuc_id_to_center = dict([(k,v) for k,v in zip(nucleu_ids,nucleus_centers)])

In [ ]:
len(G_undirected_non_multi.edges())

In [ ]:
print("hi")

In [ ]:
import matplotlib.pyplot as plt
def graph_to_soma_to_soma_distance(G):
    total_unique_edges = np.array(list(G.edges()))
    edges_mask = np.all(total_unique_edges>0,axis=1)
    soma_distances = np.array([[nuc_id_to_center[k],nuc_id_to_center[v]] for k,v in total_unique_edges[edges_mask]])
    soma_distances_norm = np.linalg.norm(soma_distances[:,0,:] - soma_distances[:,1,:],axis=1)

    
    fig,ax = plt.subplots(1,1)
    ax.hist(soma_distances_norm/1000,bins=100,density=False)
    ax.set_title("Soma to Soma Distance of Unique Direct Connections \n After Auto Proofreading")
    ax.set_xlabel("Soma to Soma Distance (um)")
    ax.set_ylabel("Counts")
    plt.yscale("log")
    plt.xscale("linear")
    plt.show()

graph_to_soma_to_soma_distance(G)

In [ ]:
graph_to_soma_to_soma_distance(G_undirected_non_multi)

In [ ]:
total_unique_edges = np.array(list(G_undirected_non_multi.edges()))


In [ ]:
edges_mask = np.all(total_unique_edges>0,axis=1)
edges_mask

In [ ]:
soma_distances = np.array([[nuc_id_to_center[k],nuc_id_to_center[v]] for k,v in total_unique_edges[edges_mask]])
soma_distances

In [ ]:
soma_distances.shape

In [ ]:
soma_distances_norm = np.linalg.norm(soma_distances[:,0,:] - soma_distances[:,1,:],axis=1)
soma_distances_norm.shape

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1)
ax.hist(soma_distances_norm/1000,bins=100,density=False)
ax.set_title("Soma to Soma Distance of Unique Direct Connections \n After Auto Proofreading")
ax.set_xlabel("Soma to Soma Distance (um)")
ax.set_ylabel("Counts")
plt.yscale("log")
plt.xscale("linear")
plt.show()

In [ ]:
alpha_level = 0.7

import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1)
ax.hist(total_distances_no_self_loops/1000,bins=100,density=False,color="blue",label="Before Auto Proofreading",alpha=alpha_level)
ax.hist(total_distances_no_self_loops_auto/1000,bins=100,density=False,color="red",label="After Auto Proofreading",alpha=alpha_level)
ax.set_title("Soma to Soma Distance of Direct Connections")
ax.set_xlabel("Soma to Soma Distance (um)")
ax.set_ylabel("Counts")

ax.set_yscale("log")
ax.set_xscale("linear")
ax.legend()
plt.show()